## 1. 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import matplotlib.pyplot as plt

## 2. 데이터셋 로드 및 전처리

In [ ]:
prices_dataset = pd.read_csv("prices.csv",header=0)
prices_dataset

In [ ]:
yahoo = prices_dataset[prices_dataset["symbol"]=='YHOO']
yahoo

In [ ]:
yahoo_stock_prices = yahoo['close'].values.astype('float32')
yahoo_stock_prices = yahoo_stock_prices.reshape(1762,1)
yahoo_stock_prices.shape

In [ ]:
yahoo_stock_prices_copy = yahoo_stock_prices.copy()

scaler = MinMaxScaler(feature_range=(0,1))

yahoo_stock_prices = scaler.fit_transform(yahoo_stock_prices)

In [ ]:
train_size = int(len(yahoo_stock_prices) * 0.80)
test_size = len(yahoo_stock_prices) - train_size
train, test = yahoo_stock_prices[0:train_size,:], yahoo_stock_prices[train_size:len(yahoo_stock_prices),:]

train_ratio = round(len(train)/len(yahoo_stock_prices),3)
test_ratio = round(len(test)/len(yahoo_stock_prices),3)
print(f"Train set 비율 : {train_ratio}, Test set 비율 : {test_ratio}")

In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
samples = trainX.shape[0]
test_samples = testX.shape[0]
time_step = 1
features = 1

trainX = np.reshape(trainX,(samples,time_step,features))
testX = np.reshape(testX, (test_samples,time_step,features))

## 3. 모델정의

In [ ]:
model = Sequential()

model.add(LSTM(50,return_sequences=True, input_shape=(time_step, features)))
model.add(Dropout(0.2))

model.add(LSTM(100,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('linear'))

model.compile(loss='mse',optimizer='rmsprop')

In [ ]:
model.summary()

## 4. 학습 및 plotting

In [ ]:
model.fit(trainX,trainY,batch_size=128,epochs=10,validation_split=0.05)

In [ ]:
def plot_results_multiple(predicted_data, true_data):
    plt.subplot(1,2,1)
    plt.title("Actual price")
    plt.plot(scaler.inverse_transform(true_data.reshape(-1,1)))

    plt.subplot(1,2,2)
    plt.title("Predicted price")
    plt.plot(scaler.inverse_transform(np.array(predicted_data)),'r')
    plt.show()

## 5. 예측 및 테스트

In [ ]:
predicted_price = model.predict(testX)
plot_results_multiple(predicted_price,testY)

In [ ]:
def predict_sequences_multiple(model, firstValue,length):
    prediction_seqs = []
    curr_frame = firstValue
    
    for i in range(length): 
        predicted = []        
        
        predicted_price = model.predict(curr_frame[np.newaxis,:,:])
        predicted.append(predicted_price[0][0])

        curr_frame = predicted_price        
        prediction_seqs.append(predicted[-1])
        
    return prediction_seqs

predict_length=5
predictions = predict_sequences_multiple(model, testX[3], predict_length)
print(scaler.inverse_transform(np.array(predictions).reshape(-1, 1)))

In [ ]:
predicted_price = scaler.inverse_transform(np.array(predictions).reshape(-1,1))
true_price = scaler.inverse_transform(testY[3:3+predict_length].reshape(-1,1))

In [ ]:
for i, [pred,true] in enumerate(zip(predicted_price,true_price)):
    print(f"{i+1}일 후, 실제 주식가격 : {true}, 예측 주식 가격 : {pred}")